# **PROCESAMIENTO DE LENGUAJE NATURAL**

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

import numpy as np

### **Carga y preprocesamiento**

En este paso, estamos preparando los datos para el entrenamiento del modelo. Extraemos los textos de los tweets, que serán nuestras características (X), y separamos las etiquetas de sentimientos (y), que el modelo intentará predecir. Las columnas ID y Tweet se eliminan del conjunto de etiquetas, ya que no son necesarias para el modelado.

In [2]:
import pandas as pd

# Cargar los archivos CSV proporcionados
train_df = pd.read_csv('sem_eval_train_es.csv')
test_df = pd.read_csv('sem_eval_test_blank_es.csv')

# Mostrar las primeras filas de cada dataframe para entender su estructura
train_df_head = train_df.head()
test_df_head = test_df.head()

# Información sobre el dataframe de entrenamiento
train_df_info = train_df.info()

train_df_head, test_df_head, train_df_info


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3561 entries, 0 to 3560
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            3561 non-null   object
 1   Tweet         3561 non-null   object
 2   anger         3561 non-null   bool  
 3   anticipation  3561 non-null   bool  
 4   disgust       3561 non-null   bool  
 5   fear          3561 non-null   bool  
 6   joy           3561 non-null   bool  
 7   love          3561 non-null   bool  
 8   optimism      3561 non-null   bool  
 9   pessimism     3561 non-null   bool  
 10  sadness       3561 non-null   bool  
 11  surprise      3561 non-null   bool  
 12  trust         3561 non-null   bool  
dtypes: bool(11), object(2)
memory usage: 94.0+ KB


(              ID                                              Tweet  anger  \
 0  2018-Es-01643  @aliciaenp Ajajjaa somos del clan twitteras pe...  False   
 1  2018-Es-05142  @AwadaNai la mala suerte del gato fichame la c...  False   
 2  2018-Es-05379  @audiomano A mí tampoco me agrado mucho eso. E...   True   
 3  2018-Es-00208  Para llevar a los bebes de un lugar a otro deb...  False   
 4  2018-Es-01385  @DalasReview me encanta la terrible hipocresia...   True   
 
    anticipation  disgust   fear    joy   love  optimism  pessimism  sadness  \
 0         False    False  False   True  False     False      False    False   
 1         False    False   True  False  False     False       True    False   
 2         False    False  False  False  False     False      False    False   
 3         False    False  False   True  False     False      False    False   
 4         False     True  False  False  False     False      False    False   
 
    surprise  trust  
 0     False  False 

### **Pasos previos: división y vectorización**

Para evaluar el rendimiento del modelo antes de aplicarlo a los datos de prueba, dividimos el conjunto de datos original en dos partes: el conjunto de entrenamiento (X_train, y_train), que contiene el 80% de los datos, y el conjunto de validación (X_val, y_val), que contiene el 20% restante. Esto nos permitirá verificar cómo generaliza el modelo a datos que no ha visto durante el entrenamiento.

In [3]:
# Separar características (tweets) y etiquetas
X = train_df['Tweet']
y = train_df.drop(columns=['ID', 'Tweet'])

# Dividir el conjunto de entrenamiento para validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un pipeline que vectorice el texto y entrene un modelo de clasificación
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),  # Convertir texto a TF-IDF
    ('clf', MultiOutputClassifier(LogisticRegression(solver='lbfgs', max_iter=1000)))
])

Aquí, configuramos un pipeline que combina dos pasos importantes:

1.   **Vectorización del Texto:** Convertimos los tweets en vectores numéricos utilizando TF-IDF, lo que nos permite cuantificar la importancia de las palabras en cada tweet.
2.   **Entrenamiento del Modelo:** Usamos un clasificador de regresión logística multietiqueta para predecir múltiples sentimientos para cada tweet.
El pipeline nos facilita la aplicación secuencial de estos pasos tanto durante el entrenamiento como durante la predicción.

### **Entrenamos...**

En esta sección, entrenamos el modelo utilizando el conjunto de entrenamiento. El pipeline previamente configurado realiza automáticamente el procesamiento del texto (vectorización TF-IDF) y el ajuste del modelo de regresión logística a los datos. Este entrenamiento permite al modelo aprender las relaciones entre los tweets y los sentimientos asociados.

Una vez que el modelo ha sido entrenado, lo aplicamos al conjunto de validación para predecir las etiquetas de sentimiento. Estas predicciones nos permitirán evaluar el rendimiento del modelo en datos nuevos y determinar si está generalizando bien o si podría estar sobreajustado a los datos de entrenamiento.



In [4]:
# Entrenar el modelo
pipeline.fit(X_train, y_train)

# Predecir en el conjunto de validación
y_pred = pipeline.predict(X_val)

# Reporte de clasificación
report = classification_report(y_val, y_pred, target_names=y.columns, output_dict=True)

report_summary = {label: report[label] for label in y.columns}
report_summary

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'anger': {'precision': 0.648936170212766,
  'recall': 0.2747747747747748,
  'f1-score': 0.38607594936708867,
  'support': 222.0},
 'anticipation': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 88.0},
 'disgust': {'precision': 0.6,
  'recall': 0.03296703296703297,
  'f1-score': 0.0625,
  'support': 91.0},
 'fear': {'precision': 1.0,
  'recall': 0.07575757575757576,
  'f1-score': 0.14084507042253522,
  'support': 66.0},
 'joy': {'precision': 0.8727272727272727,
  'recall': 0.22325581395348837,
  'f1-score': 0.3555555555555555,
  'support': 215.0},
 'love': {'precision': 0.6666666666666666,
  'recall': 0.0425531914893617,
  'f1-score': 0.08,
  'support': 47.0},
 'optimism': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 83.0},
 'pessimism': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 122.0},
 'sadness': {'precision': 0.8709677419354839,
  'recall': 0.17197452229299362,
  'f1-score': 0.28723404255319146,
  'support': 157.0

El siguiente paso es evaluar el rendimiento del modelo utilizando un reporte de clasificación. Este reporte proporciona métricas clave como la precisión, el recall y la puntuación F1 para cada etiqueta de sentimiento. Estas métricas nos ayudan a entender cuán bien el modelo está prediciendo cada sentimiento en los tweets.

* Precisión (precision): Mide la proporción de verdaderos positivos entre todos los casos predichos como positivos.

* Recall (recall): Mide la proporción de verdaderos positivos entre todos los casos que son realmente positivos.

* F1-score: Es la media armónica entre la precisión y el recall, proporcionando una única métrica para evaluar el rendimiento del modelo.

Finalmente, extraemos un resumen del reporte para cada etiqueta de sentimiento, lo que facilita el análisis de los resultados y la identificación de áreas donde el modelo podría necesitar mejoras.

### **Predecimos...**

En este código, se entrena un modelo de clasificación multietiqueta para predecir los sentimientos asociados a tweets en un conjunto de datos ciego de prueba. El proceso comienza con la configuración y entrenamiento de un pipeline que combina la vectorización de texto mediante TF-IDF y un modelo de regresión logística multietiqueta. El modelo se entrena utilizando todo el conjunto de datos disponible, que incluye los tweets y sus correspondientes etiquetas de sentimientos.

Una vez que el modelo ha sido entrenado, se aplica al conjunto ciego de prueba para predecir las probabilidades de cada sentimiento en cada tweet. Estas probabilidades indican la confianza del modelo en que un sentimiento específico está presente en un tweet. Para convertir estas probabilidades en decisiones binarias (es decir, etiquetas True o False), se define un umbral personalizado de 0.3. Esto significa que si la probabilidad de un sentimiento en un tweet es mayor o igual a 0.3, ese sentimiento se considera presente (True).

El código también incluye un mecanismo para garantizar que cada tweet tenga al menos un sentimiento clasificado como True. Si todas las etiquetas para un tweet son False, el código selecciona la etiqueta con la probabilidad más alta y la marca como True. Esto evita que un tweet quede sin ningún sentimiento asignado. Las predicciones resultantes se organizan en un DataFrame, donde se aseguran de que las etiquetas estén en formato booleano (True/False). Además, se conserva la columna ID de los tweets originales para mantener la relación entre los tweets y sus predicciones.

Finalmente, las predicciones se guardan en un archivo CSV con el nombre soluciones_jose_peiro_cardona.csv, siguiendo el formato requerido para la entrega.

In [5]:
# Entrenar el modelo usando todo el conjunto de entrenamiento
pipeline.fit(X, y)

# Predecir probabilidades en el conjunto ciego de prueba
test_proba = pipeline.predict_proba(test_df['Tweet'])

# Definir el umbral personalizado
umbral = 0.3

# Convertir las probabilidades a etiquetas binarias usando el umbral personalizado
test_pred_custom = np.array([probas[:, 1] >= umbral for probas in test_proba]).T

# Asegurar que al menos una etiqueta sea True por tweet
for i in range(test_pred_custom.shape[0]):
    if not test_pred_custom[i].any():
        # Si todas las etiquetas son False, poner True en la etiqueta con la probabilidad más alta
        test_pred_custom[i, np.argmax([probas[i, 1] for probas in test_proba])] = True

# Crear un DataFrame con las predicciones
predictions_df = pd.DataFrame(test_pred_custom, columns=y.columns)

# Agregar la columna ID desde el test_df asegurando que se mantengan los IDs originales
predictions_df.insert(0, 'ID', test_df['ID'])

# Asegurarse de que las predicciones estén en formato booleano
predictions_df = predictions_df.astype(bool)

# Mantener la columna 'ID' en su formato original (object)
predictions_df['ID'] = test_df['ID']

# Guardar las predicciones en un archivo CSV
output_filename = 'soluciones_jose_peiro_cardona.csv'
predictions_df.to_csv(output_filename, index=False)

print(f'Archivo guardado como {output_filename}')

Archivo guardado como soluciones_jose_peiro_cardona.csv
